In [1]:
!pip install dash

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
import dash
import dash_core_components as dcc
import dash_html_components as html
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import graphviz
from sklearn import tree
import base64
import plotly.figure_factory as ff
from google.colab import output
import plotly.graph_objects as go

# Load the dataset and train the decision tree model
diabetes = pd.read_csv('diabetes_binary_health_indicators_BRFSS2015.csv')
correlation_matrix = diabetes.corr().round(3)
X = diabetes.drop(["Diabetes_binary"], axis=1)
y = diabetes["Diabetes_binary"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model = DecisionTreeClassifier(max_depth=3,class_weight='balanced')
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(accuracy)
# Generate the DOT data and create the graphviz visualization
features = ['HighBP', 'HighChol', 'CholCheck', 'BMI', 'Smoker',
            'Stroke', 'HeartDiseaseorAttack', 'PhysActivity', 'Fruits', 'Veggies',
            'HvyAlcoholConsump', 'AnyHealthcare', 'NoDocbcCost', 'GenHlth',
            'MentHlth', 'PhysHlth', 'DiffWalk', 'Sex', 'Age', 'Education', 'Income'
            ]
class_names = ['0', '1']



diabetes1 = pd.read_csv('diabetes_binary_health_indicators_BRFSS2015.csv')

# Convert BMI feature to binary
diabetes1['BMI'] = diabetes1['BMI'].apply(lambda x: 1 if x >= 25 else 0)

# Split MentHlth and PhysHlth based on threshold
threshold = 14
diabetes1['MentHlth'] = diabetes1['MentHlth'].apply(lambda x: 1 if x >= threshold else 0)
diabetes1['PhysHlth'] = diabetes1['PhysHlth'].apply(lambda x: 1 if x >= threshold else 0)
diabetes1['GenHlth'] = diabetes1['GenHlth'].apply(lambda x: 1 if x >= 4 else 0)
diabetes1['Income'] = diabetes1['Income'].apply(lambda x: 1 if x >= 4 else 0)
diabetes1['Education'] = diabetes1['Education'].apply(lambda x: 1 if x >= 4 else 0)
# Initialize the Dash app
app = dash.Dash(__name__)

# Calculate the percentage of diabetes and non-diabetes for each feature
features1 = list(set(diabetes1.columns) - set(['Age','Diabetes_binary']))
diabetes_percentage = []
non_diabetes_percentage = []

for feature in features1:
    diabetes_counts = diabetes1[diabetes1[feature] == 1]['Diabetes_binary'].value_counts()
    total_counts = diabetes_counts.sum()

    diabetes_percentage.append(diabetes_counts[1] / total_counts * 100)
    non_diabetes_percentage.append(diabetes_counts[0] / total_counts * 100)

age_ranges = {
    '1-5': (1, 5),
    '6-10': (6, 10),
    '11-13': (11, 13),
    'All': (1, 13)
}


# Initialize the Dash app
app = dash.Dash(__name__)

# Define the layout
app.layout = html.Div(
    children=[
        html.H1("Diabetes Dataset Visualizations"),
                # Heatmap - Correlation Matrix
                # Bar Plot
        html.Div([
            html.H2("Diabetes vs. Non-Diabetes Percentage by Age Range"),

            # Dropdown for age range selection
            dcc.Dropdown(
                id='age-dropdown123',
                options=[
                    {'label': '18-44', 'value': '1-5'},
                    {'label': '45-69', 'value': '6-10'},
                    {'label': '70-80 or older', 'value': '11-13'},
                    {'label': 'All', 'value': '1-13'}
                ],
                value='1-5',
                clearable=False
            ),

            dcc.Graph(id='bar-plot')
        ]),
        html.Div([
            html.H2("Heatmap - Correlation Matrix by Age Range:"),

            # Dropdown for age range selection
            dcc.Dropdown(
                id='age-dropdown',
                options=[
                    {'label': '18-44', 'value': '1-5'},
                    {'label': '45-69', 'value': '6-10'},
                    {'label': '70-80 or older', 'value': '11-13'},
                    {'label': 'All', 'value': '1-13'}
                ],
                value='1-5',
                clearable=False
            ),

            dcc.Graph(id='heatmap', style={'height': '600px'})  # Increase the height of the heatmap graph
        ]),


        # Age Range Selector
        html.Div([
            html.H2("Decision Tree of diabetes by Age Range:"),
            dcc.Dropdown(
                id='age-range-dropdown',
                options=[
                    {'label': '18-44', 'value': '1-5'},
                    {'label': '45-69', 'value': '6-10'},
                    {'label': '70-80 or older', 'value': '11-13'},
                    {'label': 'All', 'value': '1-13'}
                ],
                value='1-5'
            )
        ]),

        # Decision Tree Visualization
        html.Div(id='decision-tree-container')
    ]
)


@app.callback(
    dash.dependencies.Output('bar-plot', 'figure'),
    [dash.dependencies.Input('age-dropdown123', 'value')]
)
def update_bar_plot(age_range):
    fig = go.Figure()  # Initialize the figure object
    if age_range == '1-13':
        min_age, max_age = 1, 13
    else:
        min_age, max_age = map(int, age_range.split('-'))
    #min_age, max_age = age_ranges[age_range]
    filtered_diabetes1 = diabetes1[(diabetes1['Age'] >= min_age) & (diabetes1['Age'] <= max_age)]

    offset = 0
    tick_vals = []
    for i, feature in enumerate(features1):
        tick_vals.append(i*1.80)

        unique_values = filtered_diabetes1[feature].unique()
        for value in unique_values:
            subset = filtered_diabetes1[filtered_diabetes1[feature] == value]
            diabetes_counts = subset['Diabetes_binary'].value_counts()
            total_counts = diabetes_counts.sum()
            diabetes_percentage = diabetes_counts[1] / total_counts * 100
            non_diabetes_percentage = diabetes_counts[0] / total_counts * 100

            fig.add_trace(go.Bar(x=[i + offset], y=[non_diabetes_percentage], name='Non-Diabetes',
                                 offsetgroup="Non-Diabetes", offset=0, width=0.4, base=0,marker_color='blue',
                                 ))
            fig.add_trace(go.Bar(x=[i + offset], y=[diabetes_percentage], name='Diabetes',
                                 offsetgroup="Diabetes", offset=0, width=0.4, base=0,marker_color='red',
                                ))
            offset += 0.4

    fig.update_traces(showlegend=False, legendgroup='Diabetes', selector=dict(name='Diabetes'))
    fig.update_traces(showlegend=False, legendgroup='Non-Diabetes', selector=dict(name='Non-Diabetes'))

    fig.update_layout(
        title='Diabetes vs. Non-Diabetes Percentage',
        xaxis=dict(
            tickmode='array',
            ticktext=['Eat Fruits','Not Eat Fruits' ,'High BMI','Low BMI', 'Smoker', 'Not Smoker','Do CholCheck','Didnt Do CholCheck', 'See Doctor','Dont See Doctor', 'High Education','Low Education', 'HeartDiseaseorAttack - NO','HeartDiseaseorAttack - YES' ,'Bad MentHlth', 'Good MentHlth','Low Income','High Income', "Hasnt DiffWalk",'has DiffWalk', 'Low BP','High BP', 'Had Stroke','Didnt had Stroke', 'Eat Veggies','Doesnt Eat Veggies','Feamle','Male', 'Have AnyHealthcare','Doesnt Have AnyHealthcare', 'High Chol','Low Chol', 'Bad PhysHlth','Good PhysHlth', 'Bad GenHlth','Good GenHlth', 'Low AlcoholConsump','Heavy AlcoholConsump', 'Had PhysActivity',"Hadnt PhysActivity"],
            tickvals=[0.0, 0.7, 1.8, 2.5, 3.6, 4.3, 5.4, 6.1, 7.2, 7.9, 9.0, 9.7, 10.8, 11.5, 12.6, 13.3, 14.4, 15.1, 16.2,16.9 ,18.0, 18.7, 19.8, 20.5, 21.6, 22.3, 23.4, 24.1, 25.2, 25.9, 27.0, 27.7, 28.8, 29.5, 30.6, 31.4, 32.4, 33.1, 34.2, 34.9],
            tickangle=45
        ),
        yaxis_title='Percentage',
        #barmode='group',
        width=1500,
        height=750,
        showlegend=False
    )

    return fig




@app.callback(
    dash.dependencies.Output('heatmap', 'figure'),
    [dash.dependencies.Input('age-dropdown', 'value')]
)
def update_heatmap(age_range):
    #min_age, max_age = age_ranges[age_range]
    if age_range == '1-13':
        min_age, max_age = 1, 13
    else:
        min_age, max_age = map(int, age_range.split('-'))
    filtered_diabetes = diabetes[(diabetes['Age'] >= min_age) & (diabetes['Age'] <= max_age)]
    correlation_matrix = filtered_diabetes.corr().round(3)  # Round the correlation matrix to three decimal places

    heatmap = ff.create_annotated_heatmap(
        z=correlation_matrix.values,
        x=list(correlation_matrix.columns),
        y=list(correlation_matrix.index),
        colorscale='Viridis'
    )

    heatmap.update_layout(
        title='Correlation Matrix',
        xaxis_title='Variables',
        yaxis_title='Variables'
    )

    return heatmap



# Callback function to update the decision tree visualization based on the selected age range
@app.callback(
    dash.dependencies.Output('decision-tree-container', 'children'),
    [dash.dependencies.Input('age-range-dropdown', 'value')]
)
def update_decision_tree(age_range):
    filtered_diabetes = diabetes[(diabetes['Age'] >= int(age_range.split('-')[0])) & (diabetes['Age'] <= int(age_range.split('-')[1]))]

    X_filtered = filtered_diabetes.drop(["Diabetes_binary"], axis=1)
    y_filtered = filtered_diabetes["Diabetes_binary"]

    X_train_filtered, X_test_filtered, y_train_filtered, y_test_filtered = train_test_split(X_filtered, y_filtered, test_size=0.2, random_state=42)
    model_filtered = DecisionTreeClassifier(max_depth=3,class_weight='balanced')
    model_filtered.fit(X_train_filtered, y_train_filtered)

    dot_data_filtered = tree.export_graphviz(model_filtered, out_file=None,
                                             feature_names=features,
                                             class_names=class_names,
                                             filled=True)

    graph_filtered = graphviz.Source(dot_data_filtered, format="png")
    graph_image = base64.b64encode(graph_filtered.pipe(format='png')).decode()

    return html.Img(src='data:image/png;base64,{}'.format(graph_image))


output.serve_kernel_port_as_window(8050)
if __name__ == '__main__':

  app.run_server( debug=False,dev_tools_ui=False,dev_tools_props_check=False)

0.6413986124251025


<IPython.core.display.Javascript object>

Dash is running on http://127.0.0.1:8050/



INFO:dash.dash:Dash is running on http://127.0.0.1:8050/



 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:8050
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [17/Jun/2023 18:12:42] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [17/Jun/2023 18:12:43] "GET /_dash-component-suites/dash/deps/polyfill@7.v2_10_2m1687025134.12.1.min.js HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [17/Jun/2023 18:12:43] "GET /_dash-component-suites/dash/dash-renderer/build/dash_renderer.v2_10_2m1687025134.min.js HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [17/Jun/2023 18:12:43] "GET /_dash-component-suites/dash/deps/react@16.v2_10_2m1687025134.14.0.min.js HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [17/Jun/2023 18:12:43] "GET /_dash-component-suites/dash/deps/prop-types@15.v2_10_2m1687025134.8.1.min.js HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [17/Jun/2023 18:12:43] "GET /_dash-component-suites/dash/deps/react-dom@16.v2_10_2m168702